In [ ]:
from tensorflow.keras import layers, models, Input

def build_modified_vgg16(input_shape=(224, 224, 3), num_classes=4):
    def sep_conv_block(x, filters):
        x = layers.SeparableConv2D(filters, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)
        return x

    inputs = Input(shape=input_shape)

    x = sep_conv_block(inputs, 64)
    x = sep_conv_block(x, 64)
    x = layers.MaxPooling2D()(x)

    x = sep_conv_block(x, 128)
    x = sep_conv_block(x, 128)
    x = layers.MaxPooling2D()(x)

    x = sep_conv_block(x, 256)
    res1 = sep_conv_block(x, 256)
    x = layers.add([x, res1])
    x = layers.MaxPooling2D()(x)

    x = sep_conv_block(x, 512)
    res2 = sep_conv_block(x, 512)
    x = layers.add([x, res2])
    x = layers.MaxPooling2D()(x)

    x = sep_conv_block(x, 512)
    res3 = sep_conv_block(x, 512)
    x = layers.add([x, res3])
    x = layers.MaxPooling2D()(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, x)
    return model


In [ ]:
import tensorflow as tf
import pandas as pd
import os

def load_dataset(image_dir, label_csv, class_map, batch_size=8):
    df = pd.read_csv(label_csv)
    df['label'] = df['zone_class'].map(class_map)

    def process(file, label):
        image = tf.io.read_file(file)
        image = tf.image.decode_image(image, channels=3)
        image = tf.image.resize(image, [224, 224]) / 255.0
        return image, label

    image_paths = [os.path.join(image_dir, f) for f in df['image_id']]
    labels = df['label'].values

    ds = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    ds = ds.map(process).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds


In [ ]:
import pandas as pd

class_map = {
    'urban_growth_zone': 0,
    'green_recovery_zone': 1,
    'stable_zone': 2,
    'construction_zone': 3
}
reverse_map = {v: k for k, v in class_map.items()}

train_ds = load_dataset("Files/Data/split/train/images", "results/yolov8_counts_and_class.csv", class_map)
val_ds = load_dataset("Files/Data/split/val/images", "results/yolov8_counts_and_class.csv", class_map)
test_ds = load_dataset("Files/Data/split/test/images", "results/yolov8_counts_and_class.csv", class_map)

model = build_modified_vgg16()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_ds, validation_data=val_ds, epochs=10)

# Predict on test set
preds = model.predict(test_ds)
pred_labels = preds.argmax(axis=1)

# Save predictions
test_filenames = pd.read_csv("yolov8_counts_and_class.csv")['image_id'].tail(len(pred_labels))
df = pd.DataFrame({'image_id': test_filenames, 'vgg16_zone_class': [reverse_map[p] for p in pred_labels]})
df.to_csv("vgg16_predictions.csv", index=False)


In [ ]:
import pandas as pd
import random

df = pd.read_csv("yolov8_counts_and_class.csv")
forecasts = []

for _, row in df.iterrows():
    for year in range(2025, 2031):
        forecasts.append({
            "image_id": row["image_id"],
            "year": year,
            "buildings": int(row["buildings"]) + random.randint(0, 5),
            "roads": int(row["roads"]) + random.randint(0, 3),
            "construction_zones": int(row["construction_zones"]) + random.randint(0, 2)
        })

forecast_df = pd.DataFrame(forecasts)
forecast_df.to_csv("gnn_forecast_2025_2030.csv", index=False)
